2018 / 2 / 01

Wayne Nixalo

# RetinaNet Finetuning

## 1. Imports & Initialization

In [1]:
import keras
import keras.preprocessing.image

import keras_retinanet.losses
import keras_retinanet.layers
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.models.resnet import ResNet50RetinaNet
from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.utils.image import preprocess_image, resize_image
from keras_retinanet.models.resnet import custom_objects # <-- what do I need this for?

import tensorflow as tf

import numpy as np
import os
import cv2

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

Taken from [Boring Detector train.py script](https://github.com/lexfridman/boring-detector/blob/master/train.py#L43)

Based on [initialization API from RetinaNet](https://github.com/fizyr/keras-retinanet/blob/master/keras_retinanet/models/resnet.py#L100)

In [3]:
def create_models(num_classes):
    # create "base" model (no NMS)
    image = keras.layers.Input((None, None, 3))
    
    model = ResNet50RetinaNet(image, num_classes=num_classes, weights='imagenet', nms=False)
    training_model = model
    
    # append NMS for prediction only
    classification   = model.outputs[1]
    detections       = model.outputs[2]
    boxes            = keras.layers.Lambda(lambda x: x[:, :, :4])(detections)
    detections       = keras_retinanet.layers.NonMaximumSuppression(name='nms')([boxes, classification, detections])
    prediction_model = keras.models.Model(inputs=model.inputs, outputs=model.outputs[:2] + [detections])
    
    # compile model
    training_model.compile(
        loss={
            'regression'    : keras_retinanet.losses.smooth_l1(),
            'classification':keras_retinanet.losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )
    
    return model, training_model, prediction_model

In [4]:
# keras.backend.tensorflow_backend.set_session(get_session())

# model = keras.models.load_model('data/retinanet-model/resnet50_coco_best_v1.2.2.h5',
#                                     custom_objects=custom_objects)

In [5]:
LENDATSET = 7637 # length of dataset (7637 images)
batch_size = 2

num_classes = 1
steps_per_epoch = LENDATSET // batch_size # may need edits for batch sizes that leave remainders

In [6]:
image_gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)
datagen = CSVGenerator(csv_data_file='data/interstage_retinanet_data.csv',
                       csv_class_file='data/interstage_retinanet_classes.csv',
                       image_data_generator=image_gen,
                       batch_size=batch_size)

In [7]:
model, trainer_model, predictor_model = create_models(num_classes)
# dont yet exactly know why it's done this way: 3 returned 'model' versions

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Output "concatenate_2" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "concatenate_2" during training.


## 2. Training

In [8]:
trainer_model.optimizer.lr.value()

<tf.Tensor 'Adam/lr/read:0' shape=() dtype=float32>

Oh ffs Keras just show me the fucking learning rate.

In [9]:
# # recompile training-model to change learning rate (as needed)
# trainer_model.compile(
#     loss={
#         'regression'    : keras_retinanet.losses.smooth_l1(),
#         'classification':keras_retinanet.losses.focal()
#     },
#     optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
# )

In [10]:
trainer_model.fit_generator(generator = datagen,
                            steps_per_epoch = steps_per_epoch, # eh?
                            epochs    = 1,
                            verbose   = 1,)

Epoch 1/1
  75/3818 [..............................] - ETA: 1:32:50 - loss: 4.9869 - regression_loss: 3.9088 - classification_loss: 1.0782

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 6476 (shape (400, 400, 3)) contains the following invalid boxes: [array([  6.,  47., 227., 401.,   0.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]


 139/3818 [>.............................] - ETA: 1:29:31 - loss: 4.5047 - regression_loss: 3.6945 - classification_loss: 0.8102

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 6167 (shape (400, 400, 3)) contains the following invalid boxes: [array([137., 102., 227., 401.,   0.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]


 149/3818 [>.............................] - ETA: 1:29:09 - loss: 4.4468 - regression_loss: 3.6670 - classification_loss: 0.7798

...

3688/3818 [===========================>..] - ETA: 3:05 - loss: 2.0814 - regression_loss: 1.7598 - classification_loss: 0.3216

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 3849 (shape (400, 400, 3)) contains the following invalid boxes: [array([101.,  61., 289., 402.,   0.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]


3808/3818 [============================>.] - ETA: 14s - loss: 2.0587 - regression_loss: 1.7408 - classification_loss: 0.3179

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 1433 (shape (400, 400, 3)) contains the following invalid boxes: [array([  6.,   0., 401., 373.,   0.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]


3818/3818 [==============================] - 5452s 1s/step - loss: 2.0578 - regression_loss: 1.7402 - classification_loss: 0.3176


[How to save a Keras model (or just arch, or just weights)](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model)

If training the trainer_model affects model and the prediction_model ... then that means I just have to save the weights of the trainer_model right? I can initialize them all blank like above, then load in the weights, no?

Although for usage in the detector, I need to save the entire model + weights. This is really unnecessarily confusing and makes me long for PyTorch/Fastai.

In [11]:
model.save('retinanet_model_01.h5')
trainer_model.save('retinanet_trainer_model_01.h5')
predictor_model.save('retinanet_predictor_model_01.h5')

## 3. Testing & Visualization

In [12]:
test_image_path = 'data/interstage_train/000000-000412/000001.jpg' # or script for random impath
test_image = cv2.imread(test_image_path)

prediction = predictor_model.predict_on_batch(np.expand_dims(test_image, axis=0))

In [13]:
# preprocess image for neural network
image = preprocess_image(test_image)
image, scale = resize_image(image)

# detect on image
_,_,detections = model.predict_on_batch(np.expand_dims(image, axis=0))

# compute predicted labels and scores
detected_label = np.argmax(detections[0,:,4:], axis=1)
scores         = detections[0, np.arange(detections.shape[1]), 4 + detected_label]

# correct for image scale
detections[0,:,:4] /= scale

In [14]:
bbx = detections[0, 0, :4]
bbx

array([-12.395554,  -4.907556,  17.903448,  10.321588], dtype=float32)

This isn't really working...